In [9]:
import os
import numpy as np
import keras
from keras.datasets import reuters, imdb
from keras.models import Sequential
# from keras.layers import LSTM, SimpleRNN, GRU, Dense, Dropout, Activation, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import pandas as pd

from keras.layers import LSTM, SimpleRNN, GRU, Dense, Dropout, Activation, Embedding, Conv1D, MaxPooling1D, GlobalAveragePooling1D


In [6]:



EMBEDDING_DIM = 50
# load in training/test set
data = pd.read_csv('/Users/anthonymunoz/ISTDTerm6/CDS/Computation Data Science/CDSLab9/tweets.160k.random.csv', encoding='utf-8')
data.head()

data['label'].value_counts()

vocab_size = 20000
tokenizer = Tokenizer(num_words= vocab_size)
tokenizer.fit_on_texts(data['text'])
sequences = tokenizer.texts_to_sequences(data['text'])
word_index = tokenizer.word_index
tweets = sequence.pad_sequences(sequences, padding='post', maxlen=50)


labels = data['label']
labels = labels.replace(4,1) # replace label '4' with '1' to facilitate one-hot encoding
x_train, x_test, y_train, y_test = train_test_split(tweets, labels, test_size=0.2)

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

y_train = keras.utils.to_categorical(y_train) # 2 classes
y_test = keras.utils.to_categorical(y_test)

embeddings_index = {}
GLOVE_DIR = "/Users/anthonymunoz/ISTDTerm6/CDS/Computation Data Science/CDSLab9"
f = open(os.path.join(GLOVE_DIR, 'glove.6B.50d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector





128000 train sequences
32000 test sequences
Found 400000 word vectors.


In [7]:
x_train.shape

(128000, 50)

In [30]:
model = Sequential()
model = Sequential()
model.add(Embedding(len(word_index)+1, EMBEDDING_DIM, weights=[embedding_matrix], trainable=False))
model.add(Conv1D(100, 2, activation='relu'))
model.add(Conv1D(100, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(160, 4, activation='relu'))
model.add(Conv1D(160, 5, activation='relu'))
# model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))
model.add(LSTM(256,input_shape=(x_train.shape[0],x_train.shape[1])))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=128, epochs=3, verbose=1, validation_split=0.2)

score = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 102400 samples, validate on 25600 samples
Epoch 1/3
102400/102400 [==============================] - 168s 2ms/step - loss: 0.5702 - accuracy: 0.6985 - val_loss: 0.5282 - val_accuracy: 0.7402
Epoch 2/3
102400/102400 [==============================] - 165s 2ms/step - loss: 0.5023 - accuracy: 0.7551 - val_loss: 0.5021 - val_accuracy: 0.7588
Epoch 3/3
32000/32000 [==============================] - 18s 559us/step
Test score: 0.48997454178333283
Test accuracy: 0.7594062685966492


In [51]:
# Tokenizer

EMBEDDING_DIM = 50
# load in training/test set
data = pd.read_csv('/Users/anthonymunoz/ISTDTerm6/CDS/Computation Data Science/CDSLab9/tweets.160k.random.csv', encoding='utf-8')
data.head()

data['label'].value_counts()

vocab_size = 20000
tokenizer = Tokenizer(num_words= None,char_level=True,oov_token='UNK')
tokenizer.fit_on_texts(data['text'])

alphabet = "abcdefghijklmnopqrstuvwxyz0123456789,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
print(len(alphabet))
char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1
    
tokenizer.word_index = char_dict.copy()
tokenizer.word_index[tk.oov_token] = max(char_dict.values()) + 1

sequences = tokenizer.texts_to_sequences(data['text'])
word_index = tokenizer.word_index
tweets = sequence.pad_sequences(sequences, padding='post', maxlen=50)


labels = data['label']
labels = labels.replace(4,1) # replace label '4' with '1' to facilitate one-hot encoding
x_train, x_test, y_train, y_test = train_test_split(tweets, labels, test_size=0.2)

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

y_train = keras.utils.to_categorical(y_train) # 2 classes
y_test = keras.utils.to_categorical(y_test)

vocab_size = len(tokenizer.word_index)
embedding_weights = []  # (70, 69)
embedding_weights.append(np.zeros(vocab_size))
for char, i in tokenizer.word_index.items():  # from index 1 to 69
    onehot = np.zeros(vocab_size)
    onehot[i - 1] = 1
    embedding_weights.append(onehot)

embedding_weights = np.array(embedding_weights)
print('Load')

print('Found %s word vectors.' % len(embeddings_index))

68
128000 train sequences
32000 test sequences
Load
Found 400000 word vectors.


In [50]:
model = Sequential()
model.add(Embedding(vocab_size+1, 69, weights=[embedding_weights], trainable=False))
model.add(Conv1D(100, 2, activation='relu'))
model.add(Conv1D(100, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(160, 4, activation='relu'))
model.add(Conv1D(160, 5, activation='relu'))
# model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))
model.add(LSTM(256,input_shape=(x_train.shape[0],x_train.shape[1])))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=128, epochs=3, verbose=1, validation_split=0.2)

score = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 102400 samples, validate on 25600 samples
Epoch 1/3
102400/102400 [==============================] - 159s 2ms/step - loss: 0.6050 - accuracy: 0.6649 - val_loss: 0.5469 - val_accuracy: 0.7230
Epoch 2/3
102400/102400 [==============================] - 156s 2ms/step - loss: 0.5305 - accuracy: 0.7361 - val_loss: 0.5296 - val_accuracy: 0.7387
Epoch 3/3
32000/32000 [==============================] - 16s 492us/step
Test score: 0.5123228929042816
Test accuracy: 0.7478125095367432
